In [88]:
import pandas as pd
import numpy as np
import random 
import os
import joblib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import date
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [89]:
data = pd.read_csv("/Users/yj.noh/Desktop/7day.csv", encoding = "utf-8")
data = data.sort_values(by=["brms_rider_id"])

print(data.shape) # 11,106

(11106, 27)


In [90]:
data = data.fillna('NA')

In [91]:
# 친구추천 데이터 삭제 
print(data['is_recom'].value_counts()) 
df = data

# df = data[data['is_recom'] == 0 ] 
# print(df.shape) #7,360

is_recom
0    7360
1    3746
Name: count, dtype: int64


In [92]:
# 날짜 형식 변환
df['first_available_date'] = pd.to_datetime(df['first_available_date'], format='%Y-%m-%d', errors='coerce')
df['business_start_day'] = pd.to_datetime(df['business_start_day'], format='%Y-%m-%d', errors='coerce')
df['last_working_day'] = pd.to_datetime(df['last_working_day'], format='%Y-%m-%d', errors='coerce')

## 1. first_available_date 없는 사람 / 첫 배달 가능일 존재, 실제 첫 배달 수행 여부 x 제외 

In [93]:
null_first_available_date_count = df['first_available_date'].isna().sum()
count_missing_business_start_day = df['first_available_date'].notnull() & df['business_start_day'].isnull()
count = df['last_working_day'].isna().sum()

print(f"'first_available_date'가 null인 데이터 개수: {null_first_available_date_count} 개") # 1,336
print(f"'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: {count_missing_business_start_day.sum()} 개") # 3,767
print({count}) #5114

'first_available_date'가 null인 데이터 개수: 1330 개
'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: 3774 개
{5115}


In [94]:
filtered_df = df[df['first_available_date'].notnull() & df['business_start_day'].notnull() & df['last_working_day'].notnull() ]
print(filtered_df.shape) # 5,991

(5991, 27)


## 2. 최소 한 건 이상 수행한 라이더 대상

In [95]:
print(filtered_df.isna().sum())

brms_rider_id                   0
birth                           0
delivery_method                 0
first_available_date            0
business_start_day              0
is_recom                        0
insurance_type                  0
gender                          0
recommender_rider_account_id    0
rider_no                        0
수행건수                            0
실제수행기간                          0
최다수행시각                          0
최다수행시각.1                        0
평일/주말선호                         0
2순위평일/주말                        0
최다수행요일                          0
2순위수행요일                         0
최다기상타입                          0
주요수행service_type                0
주요수행allocated_type              0
주요수행method                      0
최다수행전달거리                        0
2순위수행전달거리                       0
기본배달비_중간값                       0
총배달비_중간값                        0
last_working_day                0
dtype: int64


In [96]:
# 마지막 수행일 ~ 현재까지 기간
today = pd.to_datetime(date.today())

filtered_df['not_working'] = (today - filtered_df['last_working_day']).dt.days +1
filtered_df['첫운행이후'] = (today - filtered_df['business_start_day']).dt.days +1 

print(filtered_df.head(2))

                                          brms_rider_id  birth  \
926   0003198AAE6FE85D1F86B2D4D45F992CFAE1D3E22256C0...   1990   
2764  001E1E700EBC869D19DB54B950E506910250998C3F0446...   1990   

        delivery_method first_available_date business_start_day  is_recom  \
926                BIKE           2023-09-08         2023-09-14         0   
2764  THROTTLE_BICYCLES           2023-09-04         2023-09-08         0   

              insurance_type gender recommender_rider_account_id  \
926   BIKE_ON_DEMAND_HYUNDAI    MAN                           NA   
2764           ETC_ON_DEMAND    MAN                           NA   

                                               rider_no  ... 주요수행service_type  \
926   0003198AAE6FE85D1F86B2D4D45F992CFAE1D3E22256C0...  ...           SINGLE   
2764  001E1E700EBC869D19DB54B950E506910250998C3F0446...  ...            STACK   

     주요수행allocated_type         주요수행method 최다수행전달거리 2순위수행전달거리 기본배달비_중간값  \
926                 0.0               BIKE   

In [97]:
print(filtered_df['not_working'].value_counts())
print(filtered_df['첫운행이후'].value_counts())

not_working
2     1382
3      449
4      361
5      153
6      140
      ... 
69      12
68      10
72       7
71       6
73       2
Name: count, Length: 72, dtype: int64
첫운행이후
50    220
49    215
43    212
44    200
47    195
     ... 
16     13
8      12
7      12
6      11
2       9
Name: count, Length: 72, dtype: int64


In [98]:
def period (value):
    if value < 11:
        return '10일 이하'
    elif 11 <= value <= 20:
        return '20일 이하'
    elif 21 <= value <= 30 :
        return '30일 이하'
    elif 31 <= value <= 40 : 
        return '40일 이하'
    elif 41 <= value <= 50 :
        return '50일 이하'
    elif 51 <= value <= 60 :
        return '60일 이하'
    elif 61 <= value :
        return '61일 이상'
        
    
filtered_df['미운행기간'] = filtered_df['not_working'].apply(period)
filtered_df['첫운행이후기간'] = filtered_df['첫운행이후'].apply(period)

In [99]:
print(filtered_df['미운행기간'].value_counts())
print(filtered_df['첫운행이후기간'].value_counts())

미운행기간
10일 이하    2896
50일 이하     729
40일 이하     592
20일 이하     557
60일 이하     520
30일 이하     463
61일 이상     234
Name: count, dtype: int64
첫운행이후기간
50일 이하    1814
60일 이하    1615
61일 이상    1214
40일 이하     770
30일 이하     276
20일 이하     181
10일 이하     121
Name: count, dtype: int64


## 첫운행이후기간 ~ 미운행기간 표 

In [100]:
cross_table1 = pd.crosstab(filtered_df['미운행기간'], filtered_df['첫운행이후기간'])

# 2번째 표: 최다수행method가 'BIKE'인 데이터에 대한 교차표
filtered_df_bike = filtered_df[filtered_df['주요수행method'] == 'BIKE']
cross_table2 = pd.crosstab(filtered_df_bike['미운행기간'], filtered_df_bike['첫운행이후기간'])

# 3번째 표: 최다수행method가 'BIKE'이고 is_recom이 0인 데이터에 대한 교차표
filtered_df_bike_recom_0 = filtered_df[(filtered_df['주요수행method'] == 'BIKE') & (filtered_df['is_recom'] == 0)]
cross_table3 = pd.crosstab(filtered_df_bike_recom_0['미운행기간'], filtered_df_bike_recom_0['첫운행이후기간'])

# 결과 출력
print("1번째 표: 전체 데이터에 대한 교차표")
print(cross_table1)

print("\n2번째 표: 최다수행method가 'BIKE'인 데이터에 대한 교차표")
print(cross_table2)

print("\n3번째 표: 최다수행method가 'BIKE'이고 is_recom이 0인 데이터에 대한 교차표")
print(cross_table3)

1번째 표: 전체 데이터에 대한 교차표
첫운행이후기간  10일 이하  20일 이하  30일 이하  40일 이하  50일 이하  60일 이하  61일 이상
미운행기간                                                          
10일 이하      121     113     160     399     871     696     536
20일 이하        0      68      29      78     148     142      92
30일 이하        0       0      87      80     129      94      73
40일 이하        0       0       0     213     197     110      72
50일 이하        0       0       0       0     469     177      83
60일 이하        0       0       0       0       0     396     124
61일 이상        0       0       0       0       0       0     234

2번째 표: 최다수행method가 'BIKE'인 데이터에 대한 교차표
첫운행이후기간  10일 이하  20일 이하  30일 이하  40일 이하  50일 이하  60일 이하  61일 이상
미운행기간                                                          
10일 이하       52      59      84     220     505     372     258
20일 이하        0      19      12      38      71      64      31
30일 이하        0       0      29      26      55      31      28
40일 이하        0       0       0      70   

In [101]:
reference_date = pd.to_datetime('2023-10-11') - pd.DateOffset(days=28) #9/13

filtered_df['outcome'] = (filtered_df['last_working_day'] < reference_date).astype(int)

print(reference_date)

2023-09-13 00:00:00


In [102]:
filtered_df.to_csv('prj-ML-model-LT_OV30/on_boarding_data.csv', index= False, encoding = "utf-8")